In [4]:
!pip install spacy


In [7]:
!pip install flask

In [8]:
!pip install spacy-transformers  

   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:38
   -- ------------------------------------- 0.5/8.2 MB 204.5 kB/s eta 0:00:3

In [21]:
!pip install en_core_web_md-3.5.0.tar.gz


Processing c:\users\wajiz.pk\desktop\ai intern task\chatbot\en_core_web_md-3.5.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.2 MB 335.7 kB/s eta 0:00:35
   - -------------------------------------- 0.5/12.2 MB 335.7 kB/s eta 0:00:35
   - -------------------------------------- 0.5/12.2 MB 335.7 kB/s eta 0:00:35
   -- -------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.7.0 requires pydantic>=2.0, but you have pydantic 1.10.21 which is incompatible.
gradio 5.7.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
langchain 0.3.9 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
langchain-core 0.3.21 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
pydantic-settings 2.6.1 requires pydantic>=2.7.0, but you have pydantic 1.10.21 which is incompatible.


Load spaCy and Preprocess FAQs

In [24]:
import spacy
import json

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Use a medium model for better accuracy

# Load FAQs
with open("faqs.json", "r") as f:
    faqs = json.load(f)

# Preprocess FAQs for semantic similarity
def preprocess_faqs(faqs):
    return [(faq['question'], nlp(faq['question'])) for faq in faqs]

faq_data = preprocess_faqs(faqs)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Create a Function for Matching Questions

In [ ]:
def get_best_match(user_query, faq_data, threshold=0.7):
    query_doc = nlp(user_query)
    best_match = None
    best_score = 0

    for question, question_doc in faq_data:
        similarity = query_doc.similarity(question_doc)
        if similarity > best_score:
            best_match = question
            best_score = similarity

    if best_score > threshold:
        return best_match
    else:
        return "Sorry, I couldn't find an answer to your question."


Integrate with Flask for a Simple Web Chatbot

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/ask", methods=["POST"])
def ask():
    user_input = request.json.get("question")
    if not user_input:
        return jsonify({"answer": "Please ask a question!"})
    
    best_question = get_best_match(user_input, faq_data)
    for faq in faqs:
        if faq['question'] == best_question:
            return jsonify({"answer": faq['answer']})
    
    return jsonify({"answer": "Sorry, I couldn't find an answer."})

if __name__ == "__main__":
    app.run(debug=True)
